In [17]:
# full, 50 epoch, new std, mean

In [18]:
# !pip install torchgeo --quiet
# !pip install lightning --quiet
# !pip install prettytable

In [19]:
import torch
torch.cuda.empty_cache()

In [20]:
# os.cpu_count()

In [21]:
import os
import torch
from torch.utils.data import Dataset
import rasterio
import numpy as np
from rasterio.enums import Resampling
import torch
import torch.nn as nn
import pytorch_lightning as pl
from torchvision.models import resnet50
from torch.utils.data import DataLoader
import torchvision.transforms as T
from lightning.pytorch import Trainer
from torchvision import transforms  
from torchgeo.trainers.moco import MoCoTask
from torchgeo.models import ResNet50_Weights
import kornia.augmentation as K
import torch.nn.functional as F
import torchgeo.transforms as T
from lightning.pytorch.loggers import CSVLogger
import glob
import os
import shutil
import random
import time
from prettytable import PrettyTable
random.seed(42) 

In [22]:
# Info from metadata.json
# B1	1830 × 1830	60 m
# B2	10980 × 10980	10 m
# B3	10980 × 10980	10 m
# B4	10980 × 10980	10 m
# B5	5490 × 5490	20 m
# B6	5490 × 5490	20 m
# B7	5490 × 5490	20 m
# B8	10980 × 10980	10 m
# B8A	5490 × 5490	20 m
# B9	1830 × 1830	60 m
# B11	5490 × 5490	20 m
# B12	5490 × 5490	20 m

### Sub-sample 3k Data

In [23]:
# root_dir = "/Volumes/WD_Rabina/competition/extracted_data/s2a"
# # List all folders
# scenes = sorted(glob.glob(os.path.join(root_dir, "*/")))
# # print(scenes)
# print(len(scenes))

# no_of_files=3000

# # Randomly select 3000 scenes (without replacement)
# selected_scenes = random.sample(scenes, k=3000)

# print(f"Total selected scenes: {len(selected_scenes)}")
# # print(selected_scenes[:10])  # show first 10 for sanity check

# # Path to new folder where selected scenes will be copied
# destination_root = "data/s2a_3k_sample"
# os.makedirs(destination_root, exist_ok=True)  # create folder if it doesn't exist

# # Copy each selected folder
# count=0
# for scene_path in selected_scenes:
#     # Get folder name only (e.g., "000015")
#     folder_name = os.path.basename(os.path.normpath(scene_path))
    
#     # Destination path
#     dest_path = os.path.join(destination_root, folder_name)
#     count=count+1
#     print(count)
#     # Copy folder and all its contents
#     shutil.copytree(scene_path, dest_path)

# print(f"Copied {len(selected_scenes)} folders to {destination_root}")



### Settings

In [24]:
target_size = 224
target_batch_size=128 #prefer 256 or 128
target_num_workers=4
target_max_epoch=50
use_peft = True  
from datetime import datetime

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
logger = CSVLogger("logs", name=f"metrics_{timestamp}")

# logger = CSVLogger("logs", name=f"try_{timestamp}")

# Default
# # Similar to SimCLR: https://arxiv.org/abs/2002.05709
# aug1 = aug2 = K.AugmentationSequential(
#     K.RandomResizedCrop(size=(size, size), scale=(0.2, 1)),
#     K.RandomBrightness(brightness=(0.6, 1.4), p=0.8),
#     K.RandomContrast(contrast=(0.6, 1.4), p=0.8),
#     T.RandomGrayscale(weights=weights, p=0.2),
#     K.RandomGaussianBlur(kernel_size=(ks, ks), sigma=(0.1, 2), p=0.5),
#     K.RandomHorizontalFlip(),
#     K.RandomVerticalFlip(),  # added
#     data_keys=['input'],
# )

aug = K.AugmentationSequential(
    K.RandomResizedCrop(size=(target_size, target_size), scale=(0.4, 1.0)),
    K.RandomHorizontalFlip(),
    K.RandomVerticalFlip(),
    K.RandomGaussianBlur(kernel_size=(7,7), sigma=(0.1, 1.5), p=0.3),
    K.RandomBrightness(brightness=(0.85, 1.15), p=0.5),
    data_keys=['input'],
)

### Helper Functions

In [25]:
class SSLDataset(Dataset):
    def __init__(self, scenes, bands, transforms=None):
        """
        Args:
            scenes (list): List of scene folder paths.
            bands (list): List of band names (e.g., ["B1","B2"]).
            patch_size (tuple): Size of random crop (H, W).
            transforms (callable, optional): Optional transform to apply to patches.
        """
        self.scenes = scenes
        self.bands = bands
        # self.patch_size = patch_size
        self.transforms = transforms
        self.target_h= None
        self.target_w = None
        

        # Precompute all timestamp paths to treat each timestamp as a sample
        self.samples = []
        for scene_path in scenes:
            timestamps = sorted([
                d for d in os.listdir(scene_path)
                if os.path.isdir(os.path.join(scene_path, d))
            ])
            for ts in timestamps:
                self.samples.append(os.path.join(scene_path, ts))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        ts_path = self.samples[idx]

        with rasterio.open(os.path.join(ts_path, "B2.tif")) as src:
            target_h, target_w = src.height, src.width
            # print(target_h,target_w, "target width and height" )

        band_arrays = []

        for b in self.bands:
            path = os.path.join(ts_path, f"{b}.tif")
            with rasterio.open(path) as src:
                if src.height == target_h and src.width == target_w:
                    arr = src.read(1).astype(np.float32)
                else:
                    arr = src.read(
                        1,
                        out_shape=(target_h, target_w),
                        resampling=Resampling.bilinear
                    ).astype(np.float32)

            band_arrays.append(arr)

        # Insert fake B10
        insert_idx = 10
        b10_pad = np.zeros((target_h, target_w), dtype=np.float32)
        band_arrays.insert(insert_idx, b10_pad)

        img = np.stack(band_arrays, axis=0)

        # img_patch = self._random_crop(img)

        patch_tensor = torch.tensor(img, dtype=torch.float32)

        if self.transforms:
            patch_tensor = self.transforms(patch_tensor)

        return {"image": patch_tensor}

# def calculate_stats(dataset, n_samples=500):
#     mean = 0
#     std = 0
#     print(len(dataset))
#     n = min(len(dataset), n_samples)

#     for i in range(n):
#         sample = dataset[i]
#         img = sample["image"]   # <-- key fix

#         mean += img.mean(dim=(1, 2))
#         std += img.std(dim=(1, 2))

#     mean /= n
#     std /= n

#     return mean, std

def calculate_stats(dataset, n_samples=500):
    mean = 0
    std = 0
    total = len(dataset)
    n = min(total, n_samples)

    # Randomly choose n indices
    indices = np.random.choice(total, size=n, replace=False)
    count=0
    for i in indices:
        count=count+1
        print(count)
        sample = dataset[i]
        img = sample["image"]   # TorchGeo-style dictionary

        mean += img.mean(dim=(1, 2))
        std += img.std(dim=(1, 2))

    mean /= n
    std /= n

    return mean, std


def summary_trainable(model):
    table = PrettyTable()
    table.field_names = ["Module", "Type", "Trainable Params", "Total Params"]

    for name, module in model.named_children():
        total_params = sum(p.numel() for p in module.parameters())
        trainable_params = sum(p.numel() for p in module.parameters() if p.requires_grad)
        table.add_row([name, type(module).__name__, f"{trainable_params:,}", f"{total_params:,}"])

    total_trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
    total_params = sum(p.numel() for p in model.parameters())
    
    print(table)
    print(f"Total trainable parameters: {total_trainable:,} ({total_trainable / 1e6:.2f} M)")
    print(f"Total parameters: {total_params:,} ({total_params / 1e6:.2f} M)")


In [26]:
# root_dir = "data/s2a"
root_dir = "/kaggle/input/icpr-2026-competition-ssl-s2a-3k-subset/ICPR_SSL_S2A_3k_sample"
# List all folders
scenes = sorted(glob.glob(os.path.join(root_dir, "*/")))
# scenes = ["data/s2a/000015", "data/s2a/000016"]  # list of scene folders
bands = ["B1","B2","B3","B4","B5","B6","B7","B8","B8A","B9","B11","B12"]
# print(scenes)

In [27]:
# import time
# # One time run to get mean and std
# temp_dataset = SSLDataset(scenes, bands)
# start_time=time.time()
# mean, std = calculate_stats(temp_dataset, n_samples=10000)
# end_time=time.time()
# print(f"calculate_stats time: {(end_time-start_time)/60} min")
# print(mean)
# print(std)

In [29]:
# based on 10k samples
mean= [1333.8029, 1488.1448, 1745.9066, 1985.6210, 2322.0129, 2837.1787,
        3065.8462, 3192.4492, 3225.1826, 3344.8479,    0.0000, 2683.2991,
        2116.8357]
std = [384.9683, 472.5244, 497.7275, 590.9384, 578.0192, 641.7764, 699.6282,
        752.0769, 709.3992, 752.4539,   0.0000, 568.3574, 542.2833]

# based on 500 sample
# mean= [1041.5322, 1224.2570, 1549.6492, 1815.5840, 2171.1243, 2729.5166,
#         2990.2266, 3074.2515, 3162.9661, 3260.7983,    0.0000, 2969.8357,
#         2335.3250]
# std = [328.5996, 410.0965, 443.5781, 547.2238, 519.4624, 547.1485, 606.4136,
#         649.6067, 621.7164, 687.0721,   0.0000, 574.0366, 560.9932]

# mean = [2358.7412, 2402.7629, 2580.9255, 2614.2227, 3057.6877, 3578.1008,
#         3796.8345, 3795.6868, 3947.5913, 4833.6362,    0.0000, 3379.1743,
#         2666.4465]
# std = [2994.4861, 2847.0354, 2542.9307, 2411.1196, 2399.0249, 2137.6804,
#         2036.8357, 2042.7140, 1957.9615, 3559.4121,    0.0000, 1535.7960,
#         1393.8278]

# to avoid 0 std
std = [max(s, 1e-5) for s in std]   

# define transform
transform = transforms.Compose([
    transforms.Resize((target_size, target_size)),
    transforms.Normalize(mean=mean, std=std)
])

In [30]:
dataset = SSLDataset(scenes, bands, transforms=transform)
print(len(dataset))
print(dataset[0]['image'].shape)

15426
torch.Size([13, 224, 224])


In [31]:
data_loader = DataLoader(dataset, batch_size=target_batch_size, shuffle=True, num_workers=target_num_workers)
num_batches = len(data_loader)
print("Number of batches:", num_batches)

Number of batches: 121


In [32]:
torch.cuda.is_available()

True

In [33]:
import time
task = MoCoTask(
    model="resnet50",      
    weights= ResNet50_Weights.SENTINEL2_ALL_MOCO,
    in_channels=13,       
    version=2,             # MoCo v2
    size=target_size,          
    augmentation1=aug,
    augmentation2=aug,
    lr=1e-4,
    memory_bank_size=2048,
    temperature=0.15,
    # gather_distributed=True
    # patience=5
)

# # Load your checkpoint to resume task
# ckpt_path = "/kaggle/working/ssl_3k_ckpt_20260206_063623.ckpt"
# task = task.load_from_checkpoint(ckpt_path)

# -----------------------------
# PEFT / Full Fine-Tuning Logic
# -----------------------------
if use_peft:
    print("Using PEFT: freezing backbone except last block, training projection head...")
    for name, param in task.backbone.named_parameters():
        if "layer4" in name:      # optionally fine-tune last residual block
            # print("Layer 4 trainable")
            param.requires_grad = True
        else:
            param.requires_grad = False
else:
    print("Full fine-tuning: backbone and projection head trainable...")
    for param in task.backbone.parameters():
        param.requires_grad = True

# Momentum backbone always frozen
for param in task.backbone_momentum.parameters():
    param.requires_grad = False

# Projection head always trainable
for param in task.projection_head.parameters():
    param.requires_grad = True




# Example usage for your task
summary_trainable(task)

trainer = Trainer(
    max_epochs=target_max_epoch,
    enable_progress_bar=True, 
    log_every_n_steps=num_batches,
    precision=32,
    accelerator="gpu" if torch.cuda.is_available() else "cpu",
    logger=logger)

start_time=time.time()
trainer.fit(task, data_loader)
end_time=time.time()
print(f"Training time: {(end_time-start_time)/60} min")

/usr/local/lib/python3.12/dist-packages/torchgeo/trainers/moco.py:209: UserWarning: MoCo v2 only uses 2 layers in its projection head
  warnings.warn('MoCo v2 only uses 2 layers in its projection head')


Downloading: "https://hf.co/torchgeo/resnet50_sentinel2_all_moco/resolve/da4f3c9dbe09272eb902f3b37f46635fa4726879/resnet50_sentinel2_all_moco-df8b932e.pth" to /root/.cache/torch/hub/checkpoints/resnet50_sentinel2_all_moco-df8b932e.pth


100%|██████████| 90.1M/90.1M [00:01<00:00, 68.2MB/s]
Trainer will use only 1 of 2 GPUs because it is running inside an interactive / notebook environment. You may try to set `Trainer(devices=2)` but please note that multi-GPU inside interactive / notebook environments is considered experimental and unstable. Your mileage may vary.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
💡 Tip: For seamless cloud logging and experiment tracking, try installing [litlogger](https://pypi.org/project/litlogger/) to enable LitLogger, which logs metrics and artifacts automatically to the Lightning Experiments platform.
💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.


Using PEFT: freezing backbone except last block, training projection head...
+--------------------------+------------------------+------------------+--------------+
|          Module          |          Type          | Trainable Params | Total Params |
+--------------------------+------------------------+------------------+--------------+
|         backbone         |         ResNet         |    14,964,736    |  23,539,392  |
|    backbone_momentum     |         ResNet         |        0         |  23,539,392  |
|     projection_head      |   MoCoProjectionHead   |    26,222,848    |  26,222,848  |
| projection_head_momentum |   MoCoProjectionHead   |        0         |  26,222,848  |
|        criterion         |       NTXentLoss       |        0         |      0       |
|      augmentation1       | AugmentationSequential |        0         |      0       |
+--------------------------+------------------------+------------------+--------------+
Total trainable parameters: 41,187,584 (41.

/usr/local/lib/python3.12/dist-packages/lightning/pytorch/trainer/configuration_validator.py:70: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


┏━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━━┓
┃   ┃ Name                     ┃ Type                   ┃ Params ┃ Mode  ┃ FLOPs ┃
┡━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━╇━━━━━━━┩
│ 0 │ backbone                 │ ResNet                 │ 23.5 M │ train │     0 │
│ 1 │ backbone_momentum        │ ResNet                 │ 23.5 M │ train │     0 │
│ 2 │ projection_head          │ MoCoProjectionHead     │ 26.2 M │ train │     0 │
│ 3 │ projection_head_momentum │ MoCoProjectionHead     │ 26.2 M │ train │     0 │
│ 4 │ criterion                │ NTXentLoss             │      0 │ train │     0 │
│ 5 │ augmentation1            │ AugmentationSequential │      0 │ train │     0 │
└───┴──────────────────────────┴────────────────────────┴────────┴───────┴───────┘

Trainable params: 41.2 M                                                                                           
Non-trainable params: 58.3 M                                                                                       
Total params: 99.5 M                                                                                               
Total estimated model params size (MB): 398                                                                        
Modules in train mode: 460                                                                                         
Modules in eval mode: 0                                                                                            
Total FLOPs: 0

Output()

`Trainer.fit` stopped: `max_epochs=50` reached.


Training time: 209.16388917764027 min


In [34]:
print(task.trainer.logged_metrics)

{'train_ssl_std': tensor(0.0159), 'train_loss': tensor(3.2313)}


In [35]:
# Save the backbone encoder only
torch.save(task.backbone.state_dict(),f"ssl_backbone_{timestamp}.pth")

torch.save(task.projection_head.state_dict(), f"projection_head_{timestamp}.pth")
trainer.save_checkpoint(f"ssl_3k_ckpt_{timestamp}.ckpt")

`weights_only` was not set, defaulting to `False`.
